## Setup & Imports

In [31]:
from pathlib import Path
import pandas as pd
import xarray as xr
import yaml
import numpy as np
from itertools import zip_longest
from dask.diagnostics import ProgressBar

def load_global(config="global"):
    with open(f"configs/{config}.yaml", "r") as f:
        return yaml.safe_load(f)
    
cfg = load_global()
cfm = load_global(config="datapp_de")


## Target data

In [4]:
data_file = cfm["target_data_raw"]
df = pd.read_csv(data_file)

In [ ]:
# Create a mask: True if value is not NaN
mask = df["DE11"].notna()

# Assign a group id: whenever a NaN occurs, the group number increases
groups = mask.ne(mask.shift()).cumsum()

# Filter only groups where values are numbers, then count their sizes
lengths = df[mask].groupby(groups).size()

# Calculate the average length
average_length = lengths.mean()

In [ ]:
print("Average sequence length:", average_length)

Average sequence length: 12.860486946318568


In [ ]:
df["DED5"].fillna(0).mean()

np.float64(0.10664560958363106)

In [190]:
# filter for midnight and day
df_day = df[df['Date'].str.endswith('12:00:00')].copy()
df_day['Date'] = df_day['Date'].str[:-9]

# filter for 1980 to 2024
df_day = df_day[(df_day['Date'] >= '1980-01-01') & (df_day['Date'] <= '2024-12-31')]

## MLWP Model data

In [2]:
ds_era5 = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
ds_pangu = xr.open_zarr('gs://weatherbench2/datasets/pangu/2018-2022_0012_0p25.zarr')
ds_neuralgcm = xr.open_zarr('gs://weatherbench2/datasets/neuralgcm_deterministic/2020-512x256.zarr')
ds_graphcast = xr.open_zarr('gs://weatherbench2/datasets/graphcast/2020/date_range_2019-11-16_2021-02-01_12_hours_derived.zarr')
ds_era5_hour = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr')

NameError: name 'xr' is not defined

In [ ]:
# Check which variables are in all datasets
valid_variables1 = [v for v in ds_era5.data_vars if 
                   #v in ds_era5_hour.data_vars and
                   v in ds_pangu.data_vars and 
                   v in ds_neuralgcm.data_vars and 
                   v in ds_graphcast.data_vars]

valid_variables2 = [v for v in ds_pangu.data_vars if 
                   v in ds_era5.data_vars and 
                   #v in ds_era5_hour.data_vars and
                   #v in ds_neuralgcm.data_vars and 
                   v in ds_graphcast.data_vars]
print("          Variables present in")
print(f"{"All datasets":<{22}} {"All but neuralgcm":<{22}}")
print("-" * (22 + 22))
for a, b in zip_longest(valid_variables1, valid_variables2, fillvalue=""):
    print(f"{a:<22} {b}")

          Variables present in
All datasets           All but neuralgcm     
--------------------------------------------
geopotential           10m_u_component_of_wind
specific_humidity      10m_v_component_of_wind
temperature            10m_wind_speed
u_component_of_wind    2m_temperature
v_component_of_wind    geopotential
wind_speed             mean_sea_level_pressure
                       specific_humidity
                       temperature
                       u_component_of_wind
                       v_component_of_wind
                       wind_speed


In [ ]:
[v for v in ds_era5_hour.data_vars]

['10m_u_component_of_wind',
 '10m_v_component_of_wind',
 '2m_dewpoint_temperature',
 '2m_temperature',
 'angle_of_sub_gridscale_orography',
 'anisotropy_of_sub_gridscale_orography',
 'boundary_layer_height',
 'geopotential',
 'geopotential_at_surface',
 'high_vegetation_cover',
 'lake_cover',
 'land_sea_mask',
 'leaf_area_index_high_vegetation',
 'leaf_area_index_low_vegetation',
 'low_vegetation_cover',
 'mean_sea_level_pressure',
 'mean_surface_latent_heat_flux',
 'mean_surface_net_long_wave_radiation_flux',
 'mean_surface_net_short_wave_radiation_flux',
 'mean_surface_sensible_heat_flux',
 'mean_top_downward_short_wave_radiation_flux',
 'mean_top_net_long_wave_radiation_flux',
 'mean_top_net_short_wave_radiation_flux',
 'mean_vertically_integrated_moisture_divergence',
 'potential_vorticity',
 'sea_ice_cover',
 'sea_surface_temperature',
 'slope_of_sub_gridscale_orography',
 'snow_depth',
 'soil_type',
 'specific_humidity',
 'standard_deviation_of_filtered_subgrid_orography',
 'stan

In [ ]:
## Check dimensions and resolution of datasets

datasets = {
    "ERA5": ds_era5,
    "ERA5 1h": ds_era5_hour,
    "Pangu": ds_pangu,
    "NeuralGCM": ds_neuralgcm,
    "GraphCast": ds_graphcast,
}

# Collect info about dimensions per dataset
rows = []
for name, ds in datasets.items():
    dims = ds.sizes  # dict: {dim_name: size}
    # Try to find matching dims; if missing -> set to None
    time = dims.get("time") or dims.get("times") or None
    lat = dims.get("lat") or dims.get("latitude") or None
    lon = dims.get("lon") or dims.get("longitude") or None
    lev = dims.get("level") or dims.get("lev") or dims.get("plev") or None
    lead = dims.get("prediction_timedelta") or dims.get("lead") or None

    rows.append([name, time, lat, lon, lev, lead])

# Create DataFrame
df_dims = pd.DataFrame(rows, columns=["Dataset", "Time", "Lat", "Lon", "Lev", "Lead Time"])

# find time resolutions
for name, ds in datasets.items():
    time = ds.coords.get("time")
    if time is not None:
        deltas = np.diff(time.values)  # differences between consecutive times
        unique_deltas = np.unique(deltas)

        #from ns to hours
        unique_deltas = [int(delta / np.timedelta64(1, 'h')) for delta in unique_deltas]
        df_dims.loc[df_dims["Dataset"] == name, "Time Res/h"] = str(unique_deltas)

    lead = ds.coords.get("prediction_timedelta")
    if lead is not None:
        deltas = np.diff(lead.values)  # differences between consecutive lead times
        unique_deltas = np.unique(deltas)

        #from ns to hours
        unique_deltas = [int(delta / np.timedelta64(1, 'h')) for delta in unique_deltas]
        df_dims.loc[df_dims["Dataset"] == name, "Lead Time Res/h"] = str(unique_deltas)
 

df_dims


,Dataset,Time,Lat,Lon,Lev,Lead Time,Time Res/h,Lead Time Res/h
0,ERA5,93544,721,1440,13,NaN,[6],NaN
1,ERA5 1h,561264,721,1440,37,NaN,[1],NaN
2,Pangu,3652,721,1440,13,40.0,[12],[6]
3,NeuralGCM,797,256,512,37,31.0,[12],[12]
4,GraphCast,886,721,1440,37,40.0,[12],[6]


## Training (ERA5) data

In [2]:
ds_era5 = xr.open_zarr(cfm["era5_dataset_url"])

In [3]:
new_ds_era5 = (
    ds_era5
    .sel(latitude=slice(cfm["feature_region"]["lat_max"], cfm["feature_region"]["lat_min"]),
         longitude=slice(cfm["feature_region"]["lon_min"], cfm["feature_region"]["lon_max"]))
    [cfm["feature_variables"].keys()]
    .sel(level=cfm["feature_level"])
    .sel(time=ds_era5['time'].dt.hour == 12)
)


In [4]:
sizes_vars = {}
for var in new_ds_era5.data_vars:
    sizes_vars[var] = new_ds_era5[var].nbytes
    print(f"{var:15}: {sizes_vars[var]/1024**2:8.2f} MB")

print(f"Total size: {sum(sizes_vars.values())/1024**2:8.2f} MB")

u_component_of_wind:   356.30 MB
v_component_of_wind:   356.30 MB
temperature    :   356.30 MB
geopotential   :   356.30 MB
specific_humidity:   356.30 MB
Total size:  1781.50 MB


In [6]:
with ProgressBar():
    era5_geopotential = new_ds_era5["geopotential"].load()

[#                                       ] | 2% Completed | 360.67 ss


KeyboardInterrupt: 

In [ ]:
era5_vars = [v for v in new_ds_era5.data_vars]

era5_datasets = {}
for v in era5_vars:
    era5_datasets[v] = 

In [20]:
path = Path(cfg["data_processed_dir"])
path

WindowsPath('data/processed')

In [23]:
ds_era = xr.open_dataset(path / "era5_de_u_component_of_wind.nc")

In [33]:
era5_temperature = xr.open_dataset(path / "era5_de_temperature.nc")

In [18]:
import itertools
import matplotlib.pyplot as plt
import numpy as np

# Funktion, um die Häufigkeitsverteilung für n Würfel mit w Seiten zu berechnen
def dice_distribution(n, w):
    # Erzeuge alle möglichen Kombinationen von n Würfeln (mit Werten von 1 bis w)
    faces = range(1, w+1)
    roll_combinations = itertools.product(faces, repeat=n)

    # Berechne die Summe der Würfelergebnisse für jede Kombination
    sums = [sum(roll) for roll in roll_combinations]

    # Berechne die Häufigkeit jeder Summe
    sum_counts = np.bincount(sums, minlength=n)  # Mindestgröße ist n (minimale Summe) bis n*w (maximale Summe)

    # Erstelle die x-Achse (die möglichen Summen)
    possible_sums = range(n, n*w + 1)

    # Plot der Häufigkeitsverteilung
    plt.figure(figsize=(10, 6))
    plt.bar(possible_sums, sum_counts[n:], width=0.8, color='skyblue', edgecolor='black')
    plt.title(f'Häufigkeitsverteilung von {n}W{w}', fontsize=16)
    plt.xlabel('Summe der Würfelergebnisse', fontsize=12)
    plt.ylabel('Häufigkeit', fontsize=12)
    plt.xticks(np.arange(n, n*w + 1, step=3))
    plt.grid(True)
    plt.show()
    plt.close()

# Beispiel: 3 Würfel mit 20 Seiten
n = 10 # Anzahl der Würfel
w = 10  # Anzahl der Seiten pro Würfel
dice_distribution(n, w)


KeyboardInterrupt: 